In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import ensemble, metrics
import copy
from sklearn import preprocessing
from numpy import median
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

#### read data

In [4]:
total = pd.read_csv('total_data.csv')
fake = total[total['isFake'] == 1]
real = total[total['isFake'] == 0]

#### setting seed and output file name

In [5]:
test_seed = 8
num = 180
num_seed = 55

In [6]:
outfile = 'smote'+str(test_seed)+'_'+str(num)+'_'+str(num_seed)+'.csv'
outfile

'smote8_180_55.csv'

#### confirm the test data and sample the target to attack

In [7]:
ori_data = copy.deepcopy(total)
ori_y = ori_data['isFake']
ori_X = ori_data.drop(['isFake'],axis=1)
ori_X_train, ori_X_test, ori_y_train, ori_y_test=train_test_split(ori_X,ori_y,test_size=0.2,random_state=test_seed,stratify=ori_y)

In [8]:
s1 = set(ori_X_test.index)
s1 = np.array(list(s1))
s = s1[s1<200]
location = []
loc = list(s*8)
for i in range(len(loc)):
    for j in range(8):
        location.append(loc[i]+j)
len(location)

320

#### make a filter

In [9]:
random.seed(num_seed)
fea = random.sample(location,num)

In [10]:
att_fea_loc = []
row_index = []
for i in range(len(fea)):
    att_fea_loc.append([fea[i]//8, fea[i]%8])
    row_index.append(fea[i]//8)
row_index = set(row_index)
len(row_index)

40

In [11]:
fil = np.zeros((200, 8))
fil

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
f_t_y = copy.deepcopy(total)
f_t = f_t_y.drop('isFake',axis=1)

f_f_y = copy.deepcopy(fake)
f_f = f_f_y.drop('isFake',axis=1)

f_r_y = copy.deepcopy(real)
f_r = f_r_y.drop('isFake',axis=1)

In [13]:
scaler = MinMaxScaler()

#### define the interpolation

In [14]:
def how_close_to_real(diff,real_c):
    #random.seed(12)
    fake_c = (1-real_c)
    new_fake = []
    for i in range(len(diff)):
        if diff[i][0] == diff[i][1]:
            new_fake.append(diff[i][0])
        if abs(diff[i][0] - diff[i][1])==1:
            new_fake.append(diff[i][1])
        if abs(diff[i][0] - diff[i][1])==2:
            new_fake.append(int((diff[i][0]+diff[i][1])/2))
        if abs(diff[i][0] - diff[i][1])>2:
            if (real_c!=0):
                #diff[i] = sorted(diff[i])
                if diff[i][0] < diff[i][1] :
#                     print(round(real_c*diff[i][0]+fake_c*diff[i][1]),(diff[i][1]-1))
                    new_fake.append( random.randint( round(real_c*diff[i][0]+fake_c*diff[i][1]) , (diff[i][1]-1) ) )
                if diff[i][0] > diff[i][1] :
                    new_fake.append( random.randint( round(fake_c*diff[i][0]+real_c*diff[i][1]) , (diff[i][0]-1) ) )
            if (real_c==0):
                new_fake.append(diff[i][1])
    return new_fake

#### reverse smote attack method

In [15]:
def Smote(n_real):
#     real_seed = 4
#     n_real = 4
#     fake_seed = 6
#     n_fake = 6
    
#     def sampled_real_concat_fake(real_seed, n_real, fake_seed, n_fake):
#     np.random.seed(real_seed)
    number_of_rows = f_r_y.shape[0]
    random_indices = np.random.choice(number_of_rows, size=n_real, replace=False)
    random_rows_real = f_r_y.iloc[random_indices]

#     np.random.seed(fake_seed)
#     number_of_rows = f_f_y.shape[0]
#     random_indices = np.random.choice(number_of_rows, size=n_fake, replace=False)
    random_rows_fake = f_f_y.iloc[list(row_index)]

    concat_df = pd.concat([random_rows_real, random_rows_fake])
#     print(df)
    
#     concat_df = sampled_real_concat_fake(real_seed, n_real)
    mm_scalar = scaler.fit_transform(concat_df)
    fake_index = concat_df[concat_df['isFake']==1].index
    real_index = concat_df[concat_df['isFake']==0].index

    est = []
    fake = mm_scalar[mm_scalar[:,8] == 1]
    real = mm_scalar[mm_scalar[:,8] == 0]
    for i in range(len(fake)):
        dis = {}
        for j in range(len(real)):
            vec1 = fake[i]
            vec2 = real[j]
            dis[real_index[j]] = np.sqrt(sum(pow(vec1-vec2,2)))
            #dis[real_index[j]] = cosine_similarity([vec1], [vec2])
        dis = {k: v for k, v in sorted(dis.items(), key=lambda item: item[1])}
        est.append(list(dis.keys())[0])
    ##### est存放與各個fake account最相近的real account的index
    
    feature = dict(zip((0,1,2,3,4,5,6,7),tuple(total.columns[0:8])))
#     print(feature)
    
    all_f = []
    for i in range(len(fake_index)):
        new_fake = []
        for col in range(8):    
            if col in [0,1,2,3,6,7]:
                if fil[i][col] == 1:
                    diff_p = [[concat_df.loc[fake_index[i]][feature[col]],concat_df.loc[est[i]][feature[col]]]]
#                     print(col,diff_p)
                    n = how_close_to_real(diff_p,0.2)
#                     print(n)
                    new_fake.append(n[0])
                if fil[i][col] == 0:
                    new_fake.append(total.iloc[fake_index[i]][feature[col]])
            if col in [4,5]:
                if fil[i][col] == 1:
                    diff_p = [[concat_df.loc[fake_index[i]][feature[col]],concat_df.loc[est[i]][feature[col]]]]
#                     print(col,diff_p)
                    if diff_p[0][0] == diff_p[0][1]:
                        new_fake.append(diff_p[0][0])
                    if diff_p[0][0] != diff_p[0][1]:
                        new_fake.append(diff_p[0][1])
                if fil[i][col] == 0:
                    new_fake.append(total.iloc[fake_index[i]][feature[col]])
        all_f.append(new_fake)
        
    return all_f

#### get the adversarial sample and output a dataset with adv. sample

In [16]:
nf = Smote(len(row_index))

In [17]:
new_fake_df = pd.DataFrame(nf, columns = f_t.columns, index = list(row_index))
new_fake_df['isFake'] = [1 for i in range(len(row_index))]

In [18]:
com_1 = total.iloc[list(set(list(total.index))-set(list(new_fake_df.index)))]
com_1

,userFollowerCount,userFollowingCount,userBiographyLength,userMediaCount,userHasProfilPic,userIsPrivate,usernameDigitCount,usernameLength,isFake
0,25,1937,0,0,1,1,0,10,1
2,15,399,0,0,0,0,3,12,1
4,264,4651,0,0,1,0,0,14,1
5,33,1470,0,2,1,1,4,13,1
6,420,4883,30,8,1,0,0,12,1
...,...,...,...,...,...,...,...,...,...
1189,239,1134,145,9,1,0,0,19,0
1190,91,149,0,7,1,0,0,18,0
1191,78,783,28,1,1,0,0,11,0
1192,135,517,44,4,1,0,4,11,0


In [19]:
export = pd.concat((new_fake_df,com_1))
export = export.sort_index(axis=0)

In [20]:
# export.to_csv(outfile)